In [ ]:
import pyodbc
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Connect to the database
conn_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:finance-datacenter.database.windows.net,1433;Database=high-frequency-data;Uid=CloudSA133cd0dc;Pwd=cloud.admin2023;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

cnx = pyodbc.connect(conn_string)

# Query the database

df = pd.read_sql("SELECT date_time, price FROM [dbo].[ive_tickbidask] WHERE date_time >= '2022-01-01 00:00:00.000';", cnx, index_col='date_time', parse_dates=['date_time'])

cnx.close()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df.index, df["price"], label="Price")
plt.legend()
plt.show()

In [ ]:
import numpy as np

returns = np.array(
    [
        (df["price"][i] - df["price"][i - 1]) / df["price"][i - 1] * 100
        for i in range(1, len(df))
    ]
)
plt.figure(figsize=(20, 10))
plt.plot(df.index[1:], returns)
plt.show()

In [ ]:
sq_returns_sum = sum(returns**2)
print("Volatility: ", np.sqrt(sq_returns_sum / len(returns)) * 100, "%", sep="")

In [ ]:
dates = [df.index[0]]

for date in df.index:
    if date - dates[-1] >= pd.Timedelta("30 seconds"):
        dates.append(date)

partial_df = df[df.index.isin(dates)]
partial_df

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(partial_df.index, partial_df["price"], label="Price")
plt.legend()
plt.show()

In [ ]:
returns = np.array(
    [
        (partial_df["price"][i] - partial_df["price"][i - 1])
        / partial_df["price"][i - 1]
        * 100
        for i in range(1, len(partial_df))
    ]
)
plt.figure(figsize=(20, 10))
plt.plot(partial_df.index[1:], returns)


sq_returns_sum = sum(returns**2)
print("Volatility: ", np.sqrt(sq_returns_sum / len(returns)) * 100, "%", sep="")

plt.show()